In [2]:
import collections

In [3]:
collections.defaultdict()

defaultdict(None, {})

In [4]:
A = [G for G in graphs(6) if G.is_connected()]

In [5]:
G = graphs.CycleGraph(6)

In [6]:
G = A[9]

In [7]:
P = G.automorphism_group()

In [26]:
g = P.gens()[0]

In [30]:
[g(i) for i in [0,1,2,3]]

[0, 2, 1, 3]

In [27]:
1<<G.order()

64

In [37]:
bini(1)

(0, 0, 0, 0, 0, 1)

In [100]:
def posibleStates(G):
    bini = lambda x : tuple( [(x >> i) & 1 for i in range(G.order())] )
    
    P = G.automorphism_group()
    
    states = {bini(u):None for u in range((1<<G.order())+1)}
    
    numStates = 0
    reducedStates = {}
    for u, val in states.items():
        if val is None: # Not found early
            reducedStates[u] = numStates
            numStates += 1
            for p in P:
                newu = tuple( u[p(i)] for i in range(G.order()) )
                states[newu] = u
    
    return states, reducedStates

In [101]:
states, reducedStates = posibleStates(G)

In [139]:
F.<r> = FunctionField(QQ)
R.<p> = F[]

In [145]:
def matrixMoran(G, states):
    PM = {}
    # Moran rule
    remaining_states = [ states[ 
                                (0,) * i +
                                (1,) + 
                                (0,) * (G.order()-i-1)
                               ] for i in range(G.order())]
    while remaining_states:
        S = remaining_states.pop()
        if S not in PM:
            PM[S] = collections.defaultdict(Integer)

            wS = sum(S)
            wS = G.order() - wS + r*wS

            for u in G:
                for v in G[u]:
                    Sp = list(S)
                    Sp[v] = S[u]
                    Sp = states[tuple(Sp)]
                    PM[S][Sp] += (r if S[u] else 1)/wS * 1/G.degree(u)
                    remaining_states.append(Sp)
    return PM

In [141]:
def matrixBernoulli(G, states, reducedStates):
    remaining_states = [ s for s in reducedStates if sum(s) == 1 ]
    PBer = {}
    while remaining_states:
        S = remaining_states.pop()
        if S not in PBer:
            PBer[S] = collections.defaultdict(Integer)

            wS = sum(S)
            wS = G.order() - wS + r*wS

            for u in G:
                if S[u]:
                    Sp = list(S)
                    for v in G[u]:
                        Sp[v] = 1
                    Sp = states[tuple(Sp)]
                    PBer[S][Sp] += r/wS * p
                    PBer[S][S] += r/wS * (1-p)
                    remaining_states.append(Sp)
                else:
                    for v in G[u]:
                        Sp = list(S)
                        Sp[v] = 0
                        Sp = states[tuple(Sp)]
                        PBer[S][Sp] += 1/wS * 1/G.degree(u)
                        remaining_states.append(Sp)


In [142]:
PM

{(1,
  0,
  0,
  0,
  0,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(1, 1, 0, 0, 0, 0): r/(r + 5), (0, 0, 0, 0, 0, 0): 1/(r + 5), (1, 0, 0, 0, 0, 0): 4/(r + 5)}),
 (0,
  0,
  0,
  0,
  0,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(0, 0, 0, 0, 0, 0): 1}),
 (1,
  1,
  0,
  0,
  0,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(1, 1, 0, 0, 0, 0): (1/2*r + 3/2)/(r + 2), (1, 1, 1, 0, 0, 0): 1/2*r/(r + 2), (1, 0, 0, 0, 0, 0): 1/2/(r + 2)}),
 (1,
  1,
  1,
  0,
  0,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(1, 1, 1, 0, 0, 0): 2/3, (1, 1, 1, 1, 0, 0): 1/3*r/(r + 1), (1, 1, 0, 0, 0, 0): 1/3/(r + 1)}),
 (1,
  1,
  1,
  1,
  0,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(1, 1, 1, 1, 0, 0): (3/4*r + 1/4)/(r + 1/2), (1, 1, 1, 1, 1, 0): 1/4*r/(r + 1/2), (1, 1, 1, 0, 0, 0): 1/4/(r + 1/2)}),
 (1,
  1,
  1,
  1,
  1,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(1, 1, 1, 1, 1, 0): 4/5*r/(r + 1/5), (1, 1, 1, 1, 1, 1): 1/5*

In [143]:
PBer

{(1,
  0,
  0,
  0,
  0,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(1, 1, 1, 0, 0, 0): (r/(r + 5))*p, (1, 0, 0, 0, 0, 0): (-r/(r + 5))*p + (r + 4)/(r + 5), (0, 0, 0, 0, 0, 0): 1/(r + 5)}),
 (0,
  0,
  0,
  0,
  0,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(0, 0, 0, 0, 0, 0): 1}),
 (1,
  1,
  1,
  0,
  0,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(1, 1, 1, 1, 0, 0): (2/3*r/(r + 1))*p, (1, 1, 1, 0, 0, 0): (-2/3*r/(r + 1))*p + (r + 2/3)/(r + 1), (1, 1, 0, 0, 0, 0): 1/3/(r + 1)}),
 (1,
  1,
  0,
  0,
  0,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(1, 1, 1, 0, 0, 0): (r/(r + 2))*p, (1, 1, 0, 0, 0, 0): (-r/(r + 2))*p + (r + 3/2)/(r + 2), (1, 0, 0, 0, 0, 0): 1/2/(r + 2)}),
 (1,
  1,
  1,
  1,
  0,
  0): defaultdict(<class 'sage.rings.integer.Integer'>, {(1, 1, 1, 1, 1, 0): (1/2*r/(r + 1/2))*p, (1, 1, 1, 1, 0, 0): (-1/2*r/(r + 1/2))*p + (r + 1/4)/(r + 1/2), (1, 1, 1, 0, 0, 0): 1/4/(r + 1/2)}),
 (1,
  1,
  1,
  1,
  1,
  0): defaultdic